In [49]:
from gym import Env  # envirinment 
from gym.spaces import Discrete, Box  #used to define action and observation spaces
import random
import numpy as np

In [80]:
class gridEnv(Env):
    def __init__(self):
#       initialize possible action spaces  
        self.action_space = Box(low= np.array([0]), high = np.array([100]))
#       initialize possible observation spaces
        self.observation_space = Box(low= -100.00), high = 100.00,shape=(1, 3), dtype=np.float32)
        #define observatin space with equal bounds [-100 to 100] of shape [1,3]
#       Set start power
        self.state = 203.0
#       Set number of training run
#       in the actual training this value should be infinity
        self.training_run = 100000
        
    def step(self,action):
#       increase or decrease power depending upon requirement
        self.state += action -1
#       decrease training run by 1
        self.training_run -= 1
    
################################################################################
# matlab simulation model that
# return frequency for reward
# return p1, p2 and load for future actions
################################################################################

#       give reward based on the frequency(observation space)  
        if self.state >= 49.5 and self.state <=50.5:
            reward = 1
        else:
            reward = -1
#       check if training run is complete  
        if self.training_run <= 0:
            done = True
        else:
            done = False
            
#       Set placeholder for info
        info = {}
    
#       return the information
        return self.state, reward, done, info
        
    def render(self):
        pass
    def reset(self):
#       resetting the env
        self.state = 50 + random.randint(-3,3)
        self.training_run = 1000
        return self.state

In [81]:
states = env.observation_space.shape
actions = env.action_space.n
env  = gridEnv()
env.observation_space.sample()
episodes = 10
for episode in range(1,episodes+1):
    state = env.reset()
    done  = False
    score = 0
    
    while not done:
        env.render()
        action  =env.action_space.sample()
        n_space, reward, done, info = env.step(action)
        score += reward
#     print('Episode:{} Score: {}' .format(episode, score))

In [82]:
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [83]:
states_shape = env.observation_space.shape
actions  = env.action_space.n

In [89]:
# building model if training from beginning
# def build_model(states,actions):
#     model = Sequential()
#     model.add(Dense(20, activation="tanh", input_dim=3, kernel_initializer="uniform"))
#     model.add(Dense(20, activation="relu", kernel_initializer="uniform"))
#     model.add(Dense(1, activation="linear", kernel_initializer="uniform"))
#     return model

In [ ]:
# import pretrained model and insert it into reinforcement model
model = keras.models.load_model("./model/model.h5")

In [90]:
#build pretrained model
model = build_model(states_shape, actions)
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 24)                48        
                                                                 
 dense_24 (Dense)            (None, 24)                600       
                                                                 
 dense_25 (Dense)            (None, 2)                 50        
                                                                 
Total params: 698
Trainable params: 698
Non-trainable params: 0
_________________________________________________________________


In [91]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [92]:
# define a reinforcement learning agent
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000, window_length = 1)
    dqn = DQNAgent(model = model, memory = memory, policy = policy,
                  nb_actions = actions, nb_steps_warmup = 10, target_model_update = 1e-12)
    return dqn

In [94]:
# training model with reinforcement learning
dqn = build_agent(model, action)
dqn.compile(Adam(lr=1e-3), metrics = ['mae'])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1)

C:\Users\anup\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)

    1/10000 [..............................] - ETA: 31:13 - reward: 1.0000

C:\Users\anup\anaconda3\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\anup\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 83s 8ms/step - reward: -0.2000
10 episodes - episode_reward: -200.000 [-1000.000, 1000.000] - loss: 0.466 - mae: 3.136 - mean_q: 8.432

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 91s 9ms/step - reward: 0.2000
10 episodes - episode_reward: 200.000 [-1000.000, 1000.000] - loss: 0.517 - mae: 2.791 - mean_q: 8.310

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 98s 10ms/step - reward: -0.4000
10 episodes - episode_reward: -400.000 [-1000.000, 1000.000] - loss: 0.533 - mae: 1.899 - mean_q: 8.349

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 99s 10ms/step - reward: 0.4020
10 episodes - episode_reward: 402.000 [-1000.000, 1000.000] - loss: 0.549 - mae: 1.985 - mean_q: 8.422

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 97s 10ms/step - reward: -0.97640s - rewar
done, took 467.981 seconds
